In [ ]:
import Pkg
Pkg.add("Pkg")
Pkg.add("CSV")
Pkg.add("Plots")
Pkg.add("DataFrames")
Pkg.add("Statistics")
Pkg.add("StatsPlots")

In [ ]:
ENV["GRDIR"]=""; Pkg.build("GR")

In [ ]:
import Pkg;
Pkg.add("Pkg")
Pkg.add("GR")
Pkg.build("GR")

In [ ]:
using CSV
using Plots
using DataFrames
using Statistics
using LinearAlgebra
using StatsPlots
using Dates
using Random

In [ ]:
ems_2018 = CSV.read("../EMS_2018.csv")

In [ ]:
head(ems_2018)

In [ ]:
feature_names = names(ems_2018)
for i in 1:31
    println(string(i), "\t", string(feature_names[i]), "\t\t\t", string(eltype(ems_2018[!, i])))
end

In [ ]:
# interesting to look at initial vs final severity
# initial call type vs final call type (any outliers?)
# Activation/dispatch Response times
# histogram of incidents by date/time, borough/zipcode, policeprecinct, districts
# could try joining income with district/schooldistrict
# what is held_indicator, incident_disposition_code, reopen indicator, special event indicator, standby or transfer indicator?

In [ ]:
#split into months to run more quickly
ems_jan2020 = CSV.read("EMS_2020_1.csv")
head(ems_jan2020)

In [ ]:
feature_names = names(ems_jan2020)
for i in 1:31
    println(string(i), "\t", string(feature_names[i]), "\t\t\t", string(eltype(ems_jan2020[!, i])))
end

In [ ]:
ems_jan2020 = ems_jan2020[ems_jan2020[:VALID_DISPATCH_RSPNS_TIME_INDC] .== "Y", :]
ems_jan2020 = ems_jan2020[ems_jan2020[:VALID_INCIDENT_RSPNS_TIME_INDC] .== "Y", :]

In [ ]:
sum(ismissing(ems_jan2020[:FIRST_HOSP_ARRIVAL_DATETIME])) #Doesn't recognize missing vals yet?

In [ ]:
#count missing for each row
#histogram 
"""
INCIDENT_RESPONSE_SECONDS_QY			String
14	INCIDENT_TRAVEL_TM_SECONDS_QY			String
16	FIRST_HOSP_ARRIVAL_DATETIME			String
19	INCIDENT_DISPOSITION_CODE			Float64
Try it as output, if drags accuracy down too much then leave out 
Maybe can use for training? But dont require as input for predictions """

"""transporting patient
patient pronounced dead
cancelled
unfounded
condition corrected
treated not transported
refused medical aid
treated and transported
triaged at scene no transport
patient gone on arrival
cancelled
duplicate incident - Probably want to filter this?
unit not sent
no disposition
"""
@df ems_jan2020 corrplot([:INCIDENT_RESPONSE_SECONDS_QY, :INCIDENT_TRAVEL_TM_SECONDS_QY, :FIRST_HOSP_ARRIVAL_DATETIME, :FINAL_CALL_TYPE])

In [ ]:
?corrplot

In [ ]:
?heatmap

In [ ]:
#heatmap(randn(10,10)) use if inputs are same values in similar range
#x, y , z. x is the categories for the x axis, y is categories on y axis, z is the value/heat at each pair x,y

In [ ]:
function string_to_float(str)
    try
        parse(Float64, str)
    catch
       0.0
    end
end
ems_jan2020[:INCIDENT_RESPONSE_SECONDS_QY] = string_to_float.(ems_jan2020[:, :INCIDENT_RESPONSE_SECONDS_QY])

In [ ]:
ems_jan2020[:INCIDENT_TRAVEL_TM_SECONDS_QY] = string_to_float.(ems_jan2020[:, :INCIDENT_TRAVEL_TM_SECONDS_QY])

In [ ]:
histogram(ems_jan2020[:INCIDENT_RESPONSE_SECONDS_QY], label="Distribution of EMS response times")
xlabel!("Response time (seconds)")
ylabel!("Frequency")

In [ ]:
"""The mean squared error of the true value y and prediction pred."""
function MSE(y, pred)
    mse = sqrt(sum((y - pred).^2)/length(y))
    return mse
end

In [ ]:
#groupby zipcode to compute avg/variance in response time.
avg_rspns_time_zip = by(ems_jan2020, :ZIPCODE, :INCIDENT_RESPONSE_SECONDS_QY => mean)
var_rspns_time_zip = by(ems_jan2020, :ZIPCODE, :INCIDENT_RESPONSE_SECONDS_QY => var)

In [ ]:
@df avg_rspns_time_zip scatter(:ZIPCODE, :INCIDENT_RESPONSE_SECONDS_QY_mean)
xlabel!("zipcode")
ylabel!("average response time (seconds)")

In [ ]:
@df var_rspns_time_zip scatter(:ZIPCODE, :INCIDENT_RESPONSE_SECONDS_QY_var)
xlabel!("zipcode")
ylabel!("variance in response time")

In [ ]:
@df avg_rspns_time_zip scatter(:ZIPCODE, :INCIDENT_RESPONSE_SECONDS_QY_mean) #Want to calculate mean for each day

In [ ]:
#preprocessing - replace INCIDENT_DATETIME with month, year, time of day
ems_jan2020[:INCIDENT_DATETIME] = map(x -> DateTime(x[:INCIDENT_DATETIME], "m/d/y I:M:S p"), eachrow(ems_jan2020))

In [ ]:
ymh = [Dates.year.(ems_jan2020[:INCIDENT_DATETIME]) Dates.month.(ems_jan2020[:INCIDENT_DATETIME]) Dates.hour.(ems_jan2020[:INCIDENT_DATETIME])] 

In [ ]:
jan2020timeresp = [ems_jan2020[:INCIDENT_RESPONSE_SECONDS_QY] ems_jan2020[:ZIPCODE] ymh]

In [ ]:
#groupby time, then groupby zipcode. Then take the mean/variance of that
by(ems_jan2020, :ZIPCODE, :INCIDENT_RESPONSE_SECONDS_QY => var)
by(ems_jan2020, :ZIPCODE, by(ems_jan2020, :HOUR, :INCIDENT_RESPONSE_SECONDS_QY => mean))

In [ ]:
histogram(ems_jan2020[:INCIDENT_TRAVEL_TM_SECONDS_QY])

In [ ]:
#groupby zipcode to compute avg/variance in response time.
avg_trvl_time_zip = by(ems_jan2020, :ZIPCODE, :INCIDENT_TRAVEL_TM_SECONDS_QY => mean)
var_trvl_time_zip = by(ems_jan2020, :ZIPCODE, :INCIDENT_TRAVEL_TM_SECONDS_QY => var)

In [ ]:
@df avg_trvl_time_zip scatter(:ZIPCODE, :INCIDENT_TRAVEL_TM_SECONDS_QY_mean)
xlabel!("zipcode")
ylabel!("average travel time (seconds)")

In [ ]:
@df var_trvl_time_zip scatter(:ZIPCODE, :INCIDENT_TRAVEL_TM_SECONDS_QY_var)
xlabel!("zipcode")
ylabel!("variance in travel time")

In [ ]:
ems_2019 = CSV.read("EMS_2019_subsampled.csv")
head(ems_2019)

In [ ]:
#FIRST_HOSP_ARRIVAL_DATETIME

In [ ]:
#use incident disposition code for multiclass classification?
#predict number of incidences in a time period (maybe also pred which hospitals) to help hospitals manage resources

In [ ]:
#random forest algorithm, KNN because data doesnt look linear

In [ ]:
#in excel data->filter->column
#summarystats

In [ ]:
length(unique(ems_jan2020[:ZIPCODE]))

## Utility Functions

In [ ]:
function string_to_float(str)
    try
        parse(Float64, str)
    catch
       0.0
    end
end

In [ ]:
"Computes a onehot vector for every entry in column given a set of categories cats"
function onehot(column, cats=unique(column))
    result = zeros(size(column, 1), size(cats,1))
    for col_idx = 1:size(column,1)
        for cats_idx = 1:size(cats,1)
            comparison = column[col_idx] == cats[cats_idx]
            
            if (!ismissing(comparison) && comparison)
                result[col_idx, cats_idx] = 1
                break
            end
        end
    end
    return result
end

In [ ]:
import Dates

function parse_date(ds::AbstractString)
    fmt = Dates.DateFormat("mm/dd/yyyy HH:MM:SS")
    
    m = match(r"(.*?)\s*(AM|PM)?$"i, ds)
    d = Dates.DateTime(m.captures[1], fmt)
    ampm = uppercase(something(m.captures[2], ""))
    d + Dates.Hour(12 * +(ampm == "PM", ampm == "" || Dates.hour(d) != 12, -1))
end

function process_dates(datetimes)
    dates = parse_date.(datetimes)
    
    years = Dates.year.(dates)
    months = Dates.month.(dates)
    weeks = Dates.week.(dates)
    days = Dates.day.(dates)
    hours = Dates.hour.(dates);
   return hcat(years, months, weeks, days, hours) 
end

In [ ]:
# preprocessing
# convert dates
# onehot encoding for zipcode
# onehot encoding for initial call type
# initial severity level code - use ordinal encoding instead of real
# offset

# Feature Preprocessing
### Dates: Year, Month, Week, Day, Hour
### ZIPCODE: Onehot encoding
### InitialCallType: Onehot encoding
### Initial severity level code: Ordinal encoding
### Offset

In [ ]:
df = CSV.read("../EMS_2020_1.csv")
feature_names = names(df);

In [ ]:
sum([x==NaN for x in df[:ZIPCODE]]) #the nans are counted in the onehot encoding...

### Filter out invalid samples. _ missing samples

In [ ]:
df = df[df[:VALID_DISPATCH_RSPNS_TIME_INDC] .== "Y", :]
df = df[df[:VALID_INCIDENT_RSPNS_TIME_INDC] .== "Y", :];

df[:INCIDENT_RESPONSE_SECONDS_QY] = string_to_float.(df[:, :INCIDENT_RESPONSE_SECONDS_QY])
df[:ZIPCODE] = string.(df[:ZIPCODE]) #string.(convert.(Int, df[:ZIPCODE]))

validFloat(x) = x::Float64 >=0
validInt(x) = x::Int64 >= 0

df = df[(validFloat.(df[!, :INCIDENT_RESPONSE_SECONDS_QY])), :];
df = df[(validInt.(df[!, :FINAL_SEVERITY_LEVEL_CODE])), :];

## Encodings

In [ ]:
dates = process_dates(df[:INCIDENT_DATETIME])

#categorical, but there are many so maybe not the best option. Get supplementary data to join on zipcode.
zipcodes = unique(df[:ZIPCODE])
zips = onehot(df[:ZIPCODE], zipcodes)

#categorical
call_types = unique(df[:INITIAL_CALL_TYPE])
icalltypes = onehot(df[:INITIAL_CALL_TYPE], call_types)

#Ordinals. Does it matter if lowest is higher severity or highest is highest severity?
isevs = df[:INITIAL_SEVERITY_LEVEL_CODE];

## Get X and Y

In [ ]:
#X = hcat(dates, zips, icalltypes, isevs, ones(size(df, 1)))
X = DataFrame(YEAR = dates[:, 1], MONTH = dates[:, 2], HOUR= dates[:, 5], ZIPCODES = df[:ZIPCODE], CALLTYPES = df[:INITIAL_CALL_TYPE], SEVERITYLEVEL = df[:INITIAL_SEVERITY_LEVEL_CODE])

In [ ]:
target1 = df[:, :FINAL_SEVERITY_LEVEL_CODE]
target2 = df[:, :INCIDENT_RESPONSE_SECONDS_QY];
#data = df[:, filter(col -> (col != "FINAL_SEVERITY_LEVEL_CODE" && col != "INCIDENT_RESPONSE_SECONDS_QY"), feature_names)]

## Form Train Test Split

In [ ]:
#Now you will split the data to create training and test sets. 
train_proportion = 0.8
n = size(df, 1)
println("Size of dataset: ", string(n))

# Put the first ntrain observations in the DataFrame df into the training set, and the rest into the test set
ntrain = convert(Int, round(train_proportion*n))

# the following variable records the features of examples in the training set
train_x = X[1:ntrain,:]
# the following variable records the features of examples in the test set
test_x = X[ntrain+1:end,:]
# the following variable records the labels of examples in the training set
train_y1 = target1[1:ntrain]
train_y2 = target2[1:ntrain]
# the following variable records the labels of examples in the test set
test_y1 = target1[ntrain+1:end];
test_y2 = target2[ntrain+1:end];

### Calculating number and percentage of missing values

In [ ]:
num_missing = 0

for i in 1:size(feature_names)[1]
    feat = feature_names[i]
    if (string(eltype(df[!, i])) == "String")
        num_missing += sum((df[feat].=="nan"))
    else
        num_missing += sum(isnan.(df[feat]))
    end
end

println(num_missing)

In [ ]:
n, num_feats = size(df)

println((n, num_feats))

println(n*num_feats)

In [ ]:
println(num_missing/(n*num_feats))

## Models
### Response Time: Regression
### Final Severity Level: Ordinal/MultiClass
### Final Calltype: MultiClass

In [ ]:
names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Decision Tree (Julia)",
         "Random Forest (Julia)", "AdaBoost (Julia)", "Naive Bayes", "Linear Discriminant Analysis",
         "Quadratic Discriminant Analysis"]

model = ()
fit!(model, X, y)

#for classifiers
accuracy = sum(predict(model, X) .== y) / length(y)

#for regression
#MSE? MAE

score = score(model, Xtest, ytest)

predict_proba(model, X)
decision_function()

In [ ]:
#AutoMLPipeline seems to do all this for you

In [ ]:
import Pkg
Pkg.add("AutoMLPipeline")

In [ ]:
using AutoMLPipeline

In [ ]:
names(X)

In [ ]:
Y = target2

In [ ]:
std = SKPreprocessor("StandardScaler")
ohe = OneHotEncoder()
kohe = SKPreprocessor("OneHotEncoder")
catf = CatFeatureSelector()
numf = NumFeatureSelector()
disc = CatNumDiscriminator(5) # unique instances <= 5 are categories
pcmc = @pipeline disc |> ((catf |> ohe) + (numf |> std)) 
dfcmc = fit_transform!(pcmc,X)

In [ ]:
#list of learners and processors
sklearners()
skpreprocessors()

In [ ]:
skrf_reg = SKLearner("RandomForestRegressor")
skgb_reg =  SKLearner("GradientBoostingRegressor")
nothing #hide

In [ ]:
learners = DataFrame() 
for learner in [skrf_reg, skgb_reg]
  pcmc = @pipeline disc |> ((catf |> ohe) + (numf |> std)) |> learner
  println(learner.name)
  mean,sd,folds,err = crossvalidate(pcmc,X,Y,"accuracy_score",5)
  global learners = vcat(learners,DataFrame(name=learner.name,mean=mean,sd=sd,kfold=folds))
end;

In [ ]:
@show learners;